# **INCOMPLETED**

##  Settings and parameters: full power.
##### OR: setting gym environment with your own backtrader engine.
****
This example assumes familarity with  Backtrader conceptions. One should at least run through Quickstart tutorial:  https://www.backtrader.com/docu/quickstart/quickstart.html
 
Typical workfolw for traditional Backtrader backtesting procedure:
- Define backtrader core engine:
  
``` python
    import backtrader as bt
    import backtrader.feeds as btfeeds
    engine = bt.Cerebro()
```

- Add some starategy class, wich has been prepared in advance as backtrader base Strategy() subclass and should define  decision-making logic:
 
``` python
    engine.addstrategy(MyStrategy)
```
     
- Set broker options, such as: cash, commission, slippage, etc.:
 
``` python
    engine.setcash(100000)
    engine.setcommission(0.001)
```
    
- Add analyzers, observers, sizers, writers to own needs:
 
``` python
    engine.addobserver(bt.observers.Trades)
    engine.addobserver(bt.observers.BuySell)
    engine.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    engine.addsizer(bt.sizers.SizerFix, stake=1000)
```
     
- Define and add data feed from one or another source (live feed is possible):
 
``` python
    MyData = btfeeds.GenericCSVData(dataname=CSVfilename.csv)
    engine.addata(MyData)
```
     
- Now backtrader enigine is ready to run backtesting:
 
``` python
   results = engine.run()
```
 
- After that you can print, analyze and think on results:
 
``` python
    engine.plot()
    my_disaster = results[0].analyzers.drawdown.get_analysis()
```
    
For BT gym environment configuration, same principles apply with some major differences:
 - strategy you prepare will be subclass of base BTgymStrategy,
   wich contains specific to RL setup methods and parameters;
    
 - this startegy will not contain decision-making logic - this part will go to RL agent;
 
 - you define you data by creating BTgymDataset class instance;
 
 - you don't add data to your engine. Just pass it to environment, BTgym Server will do.
 
 - you dont run backtrader engine manually via run() method. BTGym Server will do.
 
##### That's how you do it:

In [1]:
import numpy as np

import backtrader as bt

from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

In [22]:
class MyStrategy(BTgymStrategy):
    """
    Example subclass of BT server inner computation startegy,
    jsut overrides default get_state() method.
    
    NOTE:
    There are parameters every BTgymStrategy holds:
        state_shape:   observation state shape, by convention last dimension is time embedding one;
                       one can define any shape; should match env.observation_space.shape.
        state_low,  
        state_high:    observation space state min/max values.
        drawdown_call: simplest condition to finish episode.
        dataset_stat,   
        episode_stat:  summary descriptive statistics for entire dataset and
                       current episode. Got updated by server, one may need it.
        portfolio_actions:
                       'hold', 'buy', 'sell', 'close' -  possible agent actions.
        skip_frame:     Number of environment steps to skip before returning next response,
                        e.g. if set to 10 -- agent will interact with environment every 10th episode step;
                        Every other step agent action is assumed to be 'hold'.

    Those can be accessed within strategy such as: 'self.params.state_shape' 
    and shoud be explicitly set when adding strategy to bt.Cerebro() - see below:
    """
    def get_state(self):
        """
        Returns normalized environment observation state representation
        by computing time-embedded vector
        of price gradients.
        """
        # Prepare:
        sigmoid = lambda x: 1/(1 + np.exp(x))
        
        # T is 'gamma-like' signal hyperparameter
        # for our signal to be in about [-5,+5] range before passing it to sigmoid;
        # tweak it by hand to add/remove "contrast":
        T = 2.0e+2
        
        # Get vectors of last [m] price values as [4 x m] matrix:
        X = np.row_stack((
            self.data.open.get(size=self.p.state_shape[-1]),
            self.data.low.get(size=self.p.state_shape[-1]),
            self.data.high.get(size=self.p.state_shape[-1]),
            self.data.close.get(size=self.p.state_shape[-1]),
        ))
        
        # Compute amplified gradients:
        print('X:',X.shape)
        #dX = np.asarray(np.gradient(X))# * T 
        dX = np.gradient(X)[-1] * T
        print(type(dX))
        #print(len(dX))
        
        # Squash values in [0,1]:
        return sigmoid(dX * T)
        
# Configure backtesting engine:

MyCerebro = bt.Cerebro()

# Note (again): all kwargs here go to strategy parameters dict,
# that is our responsibility to consisit observation shape / bounds with what our get_state() computes.
MyCerebro.addstrategy(
    MyStrategy,
    state_shape=(4,10),
    state_low=0,
    state_high=1,
    drawdown_call=99,
    skip_frame=10,
)

# Than everything as usual:
MyCerebro.broker.setcash(100.0)
MyCerebro.broker.setcommission(commission=0.001)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10)
MyCerebro.addanalyzer(bt.analyzers.DrawDown)


# Define dataset:
MyDataset = BTgymDataset(
    filename='../examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
    start_weekdays=[0, 1,],
    # all other left to defaults
)

# Finally:
env = BTgymEnv(
    dataset=MyDataset,
    engine=MyCerebro,
    verbose=1,
)

[2017-06-24 01:14:13,343] Custom Dataset class used.
[2017-06-24 01:14:13,344] Custom Cerebro engine used.
[2017-06-24 01:14:13,348] Environment is ready.


In [23]:
env.reset()

[2017-06-24 01:14:14,255] No running server found, starting...
[2017-06-24 01:14:14,297] Server PID: 73026
[2017-06-24 01:14:15,024] Loaded 372678 records from <../examples/data/DAT_ASCII_EURUSD_M1_2016.csv>.
[2017-06-24 01:14:15,112] Data summary:
                open           high            low          close    volume
count  372678.000000  372678.000000  372678.000000  372678.000000  372678.0
mean        1.107109       1.107198       1.107019       1.107108       0.0
std         0.024843       0.024840       0.024847       0.024844       0.0
min         1.035250       1.035470       1.035220       1.035220       0.0
25%         1.092140       1.092230       1.092040       1.092140       0.0
50%         1.113530       1.113610       1.113450       1.113530       0.0
75%         1.124710       1.124780       1.124630       1.124710       0.0
max         1.161440       1.161600       1.160770       1.161450       0.0
[2017-06-24 01:14:15,294] Server started, pinging tcp://127.0.0.1:5

X: (4, 10)
<class 'numpy.ndarray'>
X: (4, 10)
<class 'numpy.ndarray'>


array([[ 0.40131234,  0.04742587,  0.01798621,  0.23147522,  0.549834  ,
         0.5       ,  0.5       ,  0.9168273 ,  0.98201379,  0.97340301],
       [ 0.11920292,  0.02659699,  0.05732418,  0.26894142,  0.26894142,
         0.5       ,  0.90024951,  0.98201379,  0.98787157,  0.99183743],
       [ 0.05732418,  0.06913842,  0.19781611,  0.5       ,  0.549834  ,
         0.450166  ,  0.549834  ,  0.9168273 ,  0.98201379,  0.98201379],
       [ 0.00368424,  0.01798621,  0.19781611,  0.59868766,  0.59868766,
         0.35434369,  0.90024951,  0.98787157,  0.97340301,  0.97340301]])

X: (4, 10)
<class 'numpy.ndarray'>


[2017-06-24 01:14:38,878] Episode elapsed time: 0:00:23.519281.
[2017-06-24 01:14:38,881] Server is exiting.


In [24]:
env._stop_server()

[2017-06-24 01:14:38,883] Server is exiting. Exit code: None
